In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 25
dtype = tf.float64

Step 1: Read in data<br>
using TF Learn's built in function to load MNIST data to the folder data/mnist

In [3]:
mnist = input_data.read_data_sets('data/mnist', one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


Step 2: create placeholders for features and labels<br>
each image in the MNIST data is of shape 28*28 = 784<br>
therefore, each image is represented with a 1x784 tensor<br>
there are 10 classes for each image, corresponding to digits 0 - 9.<br>
Features are of the type float, and labels are of the type int<br>

In [4]:
print(mnist.test.labels.shape)

(10000, 10)


In [5]:
sample_shape = 784
nclasses = 10
X = tf.placeholder(dtype, shape=(None, sample_shape), name="X")
Y = tf.placeholder(dtype, shape=(None, nclasses), name="Y")

Step 3: create weights and bias<br>
weights and biases are initialized to 0<br>
shape of w depends on the dimension of X and Y so that Y = X * w + b<br>
shape of b depends on Y<br>

In [6]:
print(X.shape, Y.shape)
Xshape = X.get_shape()
Yshape = Y.get_shape()
w_init = tf.zeros([sample_shape, nclasses], dtype=dtype)
b_init = tf.zeros([nclasses], dtype=dtype)
w = tf.Variable(w_init, expected_shape=(Xshape[1], Yshape[1]), name='w', dtype=dtype)
b = tf.Variable(b_init, expected_shape=(Yshape[1]), name='b', dtype=dtype)

(?, 784) (?, 10)


Step 4: build model<br>
the model that returns the logits.<br>
this logits will be later passed through softmax layer<br>
to get the probability distribution of possible label of the image<br>
DO NOT DO SOFTMAX HERE<br>

In [7]:
logits = tf.matmul(X, w) + b

Step 5: define loss function<br>
use cross entropy loss of the real labels with the softmax of logits<br>
use the method:<br>
tf.nn.softmax_cross_entropy_with_logits(logits, Y)<br>
then use tf.reduce_mean to get the mean loss of the batch<br>

In [8]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
mean_loss = tf.reduce_mean(cross_entropy)
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()
#     xdata = mnist.train.images.astype(np.float64)
#     ydata = mnist.train.labels.astype(np.float64)
#     out = sess.run(mean_loss, feed_dict={X: xdata, Y: ydata})
#     print("mean_loss =", out)

Step 6: define training op<br>
using gradient descent to minimize loss

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(mean_loss)
with tf.Session() as sess:
	start_time = time.time()
	sess.run(tf.global_variables_initializer())	
	n_batches = int(mnist.train.num_examples/batch_size)
	for i in range(n_epochs): # train the model n_epochs times
		total_loss = 0
		for _ in range(n_batches):
			X_batch, Y_batch = mnist.train.next_batch(batch_size)
			_, loss_batch = sess.run([train_step, mean_loss], feed_dict={X: X_batch, Y: Y_batch})
			total_loss += loss_batch
		print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

	print('Total time: {0} seconds'.format(time.time() - start_time))

	print('Optimization Finished!') # should be around 0.35 after 25 epochs

	# test the model
	preds = tf.nn.softmax(logits)
	correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
	accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
	
	n_batches = int(mnist.test.num_examples/batch_size)
	total_correct_preds = 0
	
	for i in range(n_batches):
		X_batch, Y_batch = mnist.test.next_batch(batch_size)
		accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y: Y_batch}) 
		total_correct_preds += accuracy_batch	
	
	print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.5905615450447104
Average loss epoch 1: 0.38156567862470275
Average loss epoch 2: 0.3490515484833415
Average loss epoch 3: 0.33083556277439147
Average loss epoch 4: 0.3205349139446371
Average loss epoch 5: 0.312164553380629
Average loss epoch 6: 0.3058410420552895
Average loss epoch 7: 0.3021202035176754
Average loss epoch 8: 0.2979125394911815
Average loss epoch 9: 0.2936547649919613
Average loss epoch 10: 0.29134428926288164
Average loss epoch 11: 0.28804904655397173
Average loss epoch 12: 0.28617556682139567
Average loss epoch 13: 0.2840365333216047
Average loss epoch 14: 0.2831148899550018
Average loss epoch 15: 0.2795088722754694
Average loss epoch 16: 0.28046978068276873
Average loss epoch 17: 0.27688440836769757
Average loss epoch 18: 0.2771376294919863
Average loss epoch 19: 0.2747983599763164
Average loss epoch 20: 0.27465993189266935
Average loss epoch 21: 0.2723282515100252
Average loss epoch 22: 0.27223860732049376
Average loss epoch 23: 0.27086979675